## Data pre processing

In [ ]:
import numpy as np
obj_centre = []
obj_width = []
lines = []
lines = [line.rstrip('\n') for line in open("./cifar10_transformed/devkit/train.txt")]
obj_centre = []
obj_width = []


lines = np.asarray(lines)
print(lines[0])
for i in range(len(lines)):
    obj_centre.append([lines[i][12:15],lines[i][15:18]])
    obj_width.append(lines[i][18:])

obj_centre = np.asarray(obj_centre)
obj_width = np.asarray(obj_width)
print(obj_centre.shape)
print(obj_width.shape)


In [ ]:
import glob
import numpy as np
from scipy import misc
files = glob.glob("./cifar10_transformed/masks/*.png")
masks_reshaped = []
for i in range(len(files)):
    temp_mask = misc.imread(files[i])
    masks_reshaped.append(np.ravel(temp_mask))

In [ ]:
data = []
file_data = glob.glob("./cifar10_transformed/imgs/*.png")
for i in range(len(file_data)):
    name = file_data[i]
    data.append(misc.imread(name))

data = np.asarray(data)
print(data.shape)

In [ ]:
import numpy as np
lines = [line.rstrip('\n') for line in open("./cifar10_transformed/devkit/test.txt")]
rows = []
columns = []
width = []


lines = np.asarray(lines)
print(lines[0])
for i in range(len(lines)):
    rows.append(lines[i][12:15])
    columns.append(lines[i][15:18])
    width.append(lines[i][18:])

In [ ]:
dimensions = np.vstack([rows,columns,width])

dimensions = dimensions.transpose()
print(dimensions.shape)

dimensions_test = np.vstack([rows,columns,width])

dimensions_test = dimensions_test.transpose()
print(dimensions_test.shape)



In [ ]:
training_labels = [int(line[10:11]) for line in open("./cifar10_transformed/devkit/train.txt")]
testing_labels = [int(line[10:11]) for line in open("./cifar10_transformed/devkit/test.txt")]
dimensions_stack = np.vstack([dimensions,dimensions_test])

In [ ]:
import tensorflow as tf
x = tf.placeholder(tf.float32, shape=[None, 48,48,3])

def batch_norm(x, is_train=True, decay=0.99, epsilon=0.001):
    shape_x = x.get_shape().as_list()
    beta = tf.get_variable('beta', shape_x[-1], initializer=tf.constant_initializer(0.0))
    gamma = tf.get_variable('gamma', shape_x[-1], initializer=tf.constant_initializer(1.0))
    moving_mean = tf.get_variable('moving_mean', shape_x[-1],
                initializer=tf.constant_initializer(0.0), trainable=False)
    moving_var = tf.get_variable('moving_var', shape_x[-1],
               initializer=tf.constant_initializer(1.0), trainable=False)

    if is_train:
        mean, var = tf.nn.moments(x, np.arange(len(shape_x)-1), keep_dims=True)
        mean = tf.reshape(mean, [mean.shape.as_list()[-1]])
        #mean = tf.reshape(mean, [mean.get_shape().as_list()[-1]])
        var = tf.reshape(var, [var.shape.as_list()[-1]])

        update_moving_mean = tf.assign(moving_mean, moving_mean*decay + mean*(1-decay))
        update_moving_var = tf.assign(moving_var,
                            moving_var*decay + shape_x[0]/(shape_x[0]-1)*var*(1-decay))
        update_ops = [update_moving_mean, update_moving_var]

        with tf.control_dependencies(update_ops):
            return tf.nn.batch_normalization(x, mean, var, beta, gamma, epsilon)

    else:
        mean = moving_mean
        var = moving_var
        return tf.nn.batch_normalization(x, mean, var, beta, gamma, epsilon)

In [ ]:
# Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
import tensorflow as tf


def transformer(U, theta, out_size, name='SpatialTransformer', **kwargs):
    """Spatial Transformer Layer
    Implements a spatial transformer layer as described in [1]_.
    Based on [2]_ and edited by David Dao for Tensorflow.
    Parameters
    ----------
    U : float
        The output of a convolutional net should have the
        shape [num_batch, height, width, num_channels].
    theta: float
        The output of the
        localisation network should be [num_batch, 6].
    out_size: tuple of two ints
        The size of the output of the network (height, width)
    References
    ----------
    .. [1]  Spatial Transformer Networks
            Max Jaderberg, Karen Simonyan, Andrew Zisserman, Koray Kavukcuoglu
            Submitted on 5 Jun 2015
    .. [2]  https://github.com/skaae/transformer_network/blob/master/transformerlayer.py
    Notes
    -----
    To initialize the network to the identity transform init
    ``theta`` to :
        identity = np.array([[1., 0., 0.],
                             [0., 1., 0.]])
        identity = identity.flatten()
        theta = tf.Variable(initial_value=identity)
    """

    def _repeat(x, n_repeats):
        with tf.variable_scope('_repeat'):
            rep = tf.transpose(
                tf.expand_dims(tf.ones(shape=tf.stack([n_repeats, ])), 1), [1, 0])
            rep = tf.cast(rep, 'int32')
            x = tf.matmul(tf.reshape(x, (-1, 1)), rep)
            return tf.reshape(x, [-1])

    def _interpolate(im, x, y, out_size):
        with tf.variable_scope('_interpolate'):
            # constants
            num_batch = tf.shape(im)[0]
            height = tf.shape(im)[1]
            width = tf.shape(im)[2]
            channels = tf.shape(im)[3]

            x = tf.cast(x, 'float32')
            y = tf.cast(y, 'float32')
            height_f = tf.cast(height, 'float32')
            width_f = tf.cast(width, 'float32')
            out_height = out_size[0]
            out_width = out_size[1]
            zero = tf.zeros([], dtype='int32')
            max_y = tf.cast(tf.shape(im)[1] - 1, 'int32')
            max_x = tf.cast(tf.shape(im)[2] - 1, 'int32')

            # scale indices from [-1, 1] to [0, width/height]
            x = (x + 1.0)*(width_f-1-1e-6) / 2.0
            y = (y + 1.0)*(height_f-1-1e-6) / 2.0

            # do sampling
            x0 = tf.cast(tf.floor(x), 'int32')
            x1 = x0 + 1
            y0 = tf.cast(tf.floor(y), 'int32')
            y1 = y0 + 1

            x0 = tf.clip_by_value(x0, zero, max_x)
            x1 = tf.clip_by_value(x1, zero, max_x)
            y0 = tf.clip_by_value(y0, zero, max_y)
            y1 = tf.clip_by_value(y1, zero, max_y)
            dim2 = width
            dim1 = width*height
            base = _repeat(tf.range(num_batch)*dim1, out_height*out_width)
            base_y0 = base + y0*dim2
            base_y1 = base + y1*dim2
            idx_a = base_y0 + x0
            idx_b = base_y1 + x0
            idx_c = base_y0 + x1
            idx_d = base_y1 + x1

            # use indices to lookup pixels in the flat image and restore
            # channels dim
            im_flat = tf.reshape(im, tf.stack([-1, channels]))
            im_flat = tf.cast(im_flat, 'float32')
            Ia = tf.gather(im_flat, idx_a)
            Ib = tf.gather(im_flat, idx_b)
            Ic = tf.gather(im_flat, idx_c)
            Id = tf.gather(im_flat, idx_d)

            # and finally calculate interpolated values
            x0_f = tf.cast(x0, 'float32')
            x1_f = tf.cast(x1, 'float32')
            y0_f = tf.cast(y0, 'float32')
            y1_f = tf.cast(y1, 'float32')
            wa = tf.expand_dims(((x1_f-x) * (y1_f-y)), 1)
            wb = tf.expand_dims(((x1_f-x) * (y-y0_f)), 1)
            wc = tf.expand_dims(((x-x0_f) * (y1_f-y)), 1)
            wd = tf.expand_dims(((x-x0_f) * (y-y0_f)), 1)
            output = tf.add_n([wa*Ia, wb*Ib, wc*Ic, wd*Id])
            return output

    def _meshgrid(height, width):
        with tf.variable_scope('_meshgrid'):
            # This should be equivalent to:
            #  x_t, y_t = np.meshgrid(np.linspace(-1, 1, width),
            #                         np.linspace(-1, 1, height))
            #  ones = np.ones(np.prod(x_t.shape))
            #  grid = np.vstack([x_t.flatten(), y_t.flatten(), ones])
            x_t = tf.matmul(tf.ones(shape=tf.stack([height, 1])),
                            tf.transpose(tf.expand_dims(tf.linspace(-1.0, 1.0, width), 1), [1, 0]))
            y_t = tf.matmul(tf.expand_dims(tf.linspace(-1.0, 1.0, height), 1),
                            tf.ones(shape=tf.stack([1, width])))

            x_t_flat = tf.reshape(x_t, (1, -1))
            y_t_flat = tf.reshape(y_t, (1, -1))

            ones = tf.ones_like(x_t_flat)
            grid = tf.concat(axis=0, values=[x_t_flat, y_t_flat, ones])
            return grid

    def _transform(theta, input_dim, out_size):
        with tf.variable_scope('_transform'):
            num_batch = tf.shape(input_dim)[0]
            height = tf.shape(input_dim)[1]
            width = tf.shape(input_dim)[2]
            num_channels = tf.shape(input_dim)[3]
            theta = tf.reshape(theta, (-1, 2, 3))
            theta = tf.cast(theta, 'float32')

            # grid of (x_t, y_t, 1), eq (1) in ref [1]
            height_f = tf.cast(height, 'float32')
            width_f = tf.cast(width, 'float32')
            out_height = out_size[0]
            out_width = out_size[1]
            grid = _meshgrid(out_height, out_width)
            grid = tf.expand_dims(grid, 0)
            grid = tf.reshape(grid, [-1])
            grid = tf.tile(grid, tf.stack([num_batch]))
            grid = tf.reshape(grid, tf.stack([num_batch, 3, -1]))

            # Transform A x (x_t, y_t, 1)^T -> (x_s, y_s)
            T_g = tf.matmul(theta, grid)
            x_s = tf.slice(T_g, [0, 0, 0], [-1, 1, -1])
            y_s = tf.slice(T_g, [0, 1, 0], [-1, 1, -1])
            x_s_flat = tf.reshape(x_s, [-1])
            y_s_flat = tf.reshape(y_s, [-1])

            input_transformed = _interpolate(
                input_dim, x_s_flat, y_s_flat,
                out_size)

            output = tf.reshape(
                input_transformed, tf.stack([num_batch, out_height, out_width, num_channels]))
            return output

    with tf.variable_scope(name):
        output = _transform(theta, U, out_size)
        return output

## network architecture

In [ ]:
import tensorflow as tf
from tensorflow.contrib.layers import flatten
from tensorflow.contrib.layers.python.layers import batch_norm as batch_norm
slim = tf.contrib.slim
mu = 0
sigma = 0.1

img_index = 0
def cifarNet(x, y, dims, is_train=True):
    two = tf.constant(2,tf.float32)
    one = tf.constant(1,tf.float32)
    vs = tf.get_variable_scope()
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 3, 32), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(32))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='SAME') + conv1_b
    conv1 = batch_norm(conv1)
    conv1 = tf.nn.relu(conv1)
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    #Layer 2
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 32, 64), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(64))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='SAME') + conv2_b
    conv2 = batch_norm(conv2)
    conv2 = tf.nn.relu(conv2)
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    #Layer 3
    conv3_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 64, 128), mean = mu, stddev = sigma))
    conv3_b = tf.Variable(tf.zeros(128))
    conv3   = tf.nn.conv2d(conv2, conv3_W, strides=[1, 1, 1, 1], padding='SAME') + conv3_b
    
    conv3 = batch_norm(conv3)
    conv3 = tf.nn.relu(conv3)
    conv3 = tf.nn.max_pool(conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    #Layer 4
    
    conv4_W = tf.Variable(tf.truncated_normal(shape=(3, 3, 128, 256), mean = mu, stddev = sigma))
    conv4_b = tf.Variable(tf.zeros(256))
    
    conv4   = tf.nn.conv2d(conv3, conv4_W, strides=[1, 1, 1, 1], padding='SAME') + conv4_b
    conv4 = batch_norm(conv4)
    
    conv4 = tf.nn.relu(conv4)
    
    #Layer 5 intermediate layer
    conv5_W = tf.Variable(tf.truncated_normal(shape=(3, 3, 256, 256), mean = mu, stddev = sigma))
    conv5_b = tf.Variable(tf.zeros(256))
    
    conv5   = tf.nn.conv2d(conv4, conv5_W, strides=[1, 1, 1, 1], padding='SAME') + conv5_b
    conv5 = batch_norm(conv5)
    conv5 = tf.nn.relu(conv5)
    
    #Layer 6 for classification
    conv6_W = tf.Variable(tf.truncated_normal(shape=(1, 1, 256, 1), mean = mu, stddev = sigma))
    conv6_b = tf.Variable(tf.zeros(1))
    
    conv6   = tf.nn.conv2d(conv5, conv6_W,strides=[1, 1, 1, 1], padding='SAME') + conv6_b
    conv6 =  flatten(conv6)
   
    bool_indices = tf.not_equal(y, two)
    threshold_indices = tf.where(bool_indices)
    ground_truth = tf.gather_nd(y, threshold_indices)
    predicted = tf.gather_nd(conv6,threshold_indices)
    
    
    
    #Layer 7 for regression
    conv7_W = tf.Variable(tf.truncated_normal(shape=(1, 1, 256, 3), mean = mu, stddev = sigma))
    conv7_b = tf.Variable([24,24,32], dtype=tf.float32)
    
    conv7   = tf.nn.conv2d(conv5, conv7_W,strides=[1, 1, 1, 1], padding='SAME') + conv7_b

    
    bool_indices_ones = tf.equal(y, one)
    threshold_indices_ones = tf.where(bool_indices_ones)
    
    conv7 = tf.reshape(conv7,[100,36,3])
    predicted_rows = tf.slice(conv7,[0,0,0], [100,36,1])
    predicted_rows = tf.reshape(predicted_rows,[100,36])


    predicted_columns = tf.slice(conv7,[0,0,1],[100,36,1])
    predicted_columns = tf.reshape(predicted_columns,[100,36])

    
    predicted_width = tf.slice(conv7,[0,0,2],[100,36,1])
    predicted_width = tf.reshape(predicted_width,[100,36])

    
    predicted_rows_dimensions = tf.gather_nd(predicted_rows, threshold_indices_ones)
    predicted_columns_dimensions = tf.gather_nd(predicted_columns, threshold_indices_ones)
    predicted_width_dimensions = tf.gather_nd(predicted_width, threshold_indices_ones)
    
    anchor_size = tf.constant(32,dtype=tf.float32)
    
    predicted_rows_dimensions = predicted_rows_dimensions / anchor_size
    predicted_rows_dimensions = tf.reshape(predicted_rows_dimensions,[-1,1])
    
    predicted_columns_dimensions = predicted_columns_dimensions / anchor_size
    predicted_columns_dimensions = tf.reshape(predicted_columns_dimensions,[-1,1])
    
    
    predicted_width_dimensions = predicted_width_dimensions / anchor_size
    predicted_width_dimensions = tf.reshape(predicted_width_dimensions,[-1,1])
    
    
    row_dims = tf.slice(dims,[0,0],[100, 1])
    row_dims = tf.tile(row_dims,[1,36])
    rows_dims = tf.gather_nd(row_dims, threshold_indices_ones)
    rows_dims = rows_dims / anchor_size
    rows_dims = tf.reshape(rows_dims,[-1,1])
    
    
    column_dims = tf.slice(dims,[0,1],[100, 1])
    column_dims = tf.tile(column_dims,[1,36])
    columns_dims = tf.gather_nd(column_dims, threshold_indices_ones)
    columns_dims = columns_dims / anchor_size
    columns_dims = tf.reshape(columns_dims,[-1,1])
    
    w_dims = tf.slice(dims,[0,2],[100, 1])
    w_dims = tf.tile(w_dims,[1,36])
    width_dims = tf.gather_nd(w_dims, threshold_indices_ones)
    width_dims = width_dims / anchor_size
    width_dims = tf.reshape(width_dims,[-1,1])
    
    ground_dims = tf.concat([rows_dims, columns_dims,width_dims], axis=1)
    

    predicted_dimensions = tf.concat([predicted_rows_dimensions, predicted_columns_dimensions, predicted_width_dimensions],axis=1)

    
    ##Object classifier


    fc0   = flatten(conv4)

    fc1_W = tf.Variable(tf.truncated_normal(shape=(9216, 256), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(256))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b

    fc1 = batch_norm(fc1)

    fc1 = tf.nn.relu(fc1)
    
    fc2_W = tf.Variable(tf.truncated_normal(shape=(256, 10), mean = mu, stddev = sigma))
    fc2_b = tf.Variable(tf.zeros(10))
    fc2   = tf.matmul(fc1, fc2_W) + fc2_b
    logits = fc2
    
    
    return conv6, ground_truth,predicted, predicted_dimensions, ground_dims,logits
    

In [ ]:
from sklearn.utils import shuffle
import numpy as np

BATCH_SIZE = 100
EPOCHS = 1500
rate_initial = 0.001
is_train = True
batch = tf.Variable(0)
y = tf.placeholder(tf.float32, shape=[None,36])
w = tf.placeholder(tf.float32, shape=[None,6,6])
dims = tf.placeholder(tf.float32, shape=[None,3])
ground_dimensions = tf.placeholder(tf.float32, shape=[None,6,3])

pred_label = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(pred_label, 10)

logits, ground_truth ,predicted, predicted_dimensions,ground_dims, logits_class = cifarNet(x, y, dims, is_train)

predicted_temp = tf.nn.sigmoid(predicted)
predicted_threshold = tf.greater(predicted_temp, 0.5)
predicted_threshold = tf.cast(predicted_threshold, tf.int32)
ground_truth_acc = tf.nn.sigmoid(ground_truth)
predicted_threshold = tf.cast(predicted_threshold, tf.float32)


cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=ground_truth, logits=predicted)

dimensions_loss = tf.losses.huber_loss(predicted_dimensions, ground_dims)


cross_entropy_classification = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits_class)
classification_loss = tf.reduce_mean(cross_entropy_classification)


loss_operation = tf.reduce_mean(cross_entropy + dimensions_loss + classification_loss)
rate = tf.train.exponential_decay(rate_initial, batch*BATCH_SIZE,
                                           20, 0.05, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

correct_prediction = tf.equal(predicted_dimensions, ground_dims)
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.int64))

correct_prediction_class = tf.equal(tf.argmax(logits_class, 1), tf.argmax(one_hot_y, 1))
accuracy_operation_class = tf.reduce_mean(tf.cast(correct_prediction_class, tf.float32))


saver = tf.train.Saver()
class_loss = []
dims_loss = []
spatial_loss = []
total_loss = []
imgs = 0
print("just before error")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = 10000
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        accuracy = 0
        X_train, y_train, d_train, l_train = shuffle(data[:10000], masks_reshaped[:10000], dimensions, training_labels)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y, batch_d, batch_l= X_train[offset:end], y_train[offset:end], d_train[offset:end], l_train[offset:end]
            
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, dims:batch_d, pred_label:batch_l})
            
            
            
        print("EPOCH {} ...".format(i+1))

        print("total loss")
        temp_loss = sess.run(loss_operation,feed_dict={x: batch_x, y: batch_y, dims:batch_d, pred_label:batch_l})
        print(temp_loss)
        total_loss.append(temp_loss)
        
        temp_dims_loss = sess.run(dimensions_loss,feed_dict={x: batch_x, y: batch_y, dims:batch_d, pred_label:batch_l})
        dims_loss.append(temp_dims_loss)
        
        temp_cross_loss = sess.run(cross_entropy,feed_dict={x: batch_x, y: batch_y, dims:batch_d, pred_label:batch_l})
        class_loss.append(temp_cross_loss)
        
        temp_spatial_loss = sess.run(classification_loss,feed_dict={x: batch_x, y: batch_y, dims:batch_d, pred_label:batch_l})
        spatial_loss.append(temp_spatial_loss)
        
        print()

    testing_accuracy = 0
    for offset in range(10000, num_examples*2, BATCH_SIZE):
        batch_x, batch_y, batch_d, batch_l= data[offset:end], masks_reshaped[offset:end], dimensions_stack[offset:end], testing_labels[offset:end]
        testing_accuracy = testing_accuracy + sess.run(accuracy_operation_class, feed_dict={x: batch_x, y: batch_y, dims:batch_d, pred_label:batch_l})
    
    print("testing accuracy is: ", (testing_accuracy/100))
    saver.save(sess, './q22Model')
    print("Model saved")